# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [1]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [2]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
!pip install keras
!pip install opencv-python
!pip install tensorflow-cpu

以下のsolution関数のみ編集してください。

In [3]:
def solution(x_test_df, train_df):

    # ライブラリをロード
    import numpy as np
    import cv2
    import os
    import random
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.utils import to_categorical
    from sklearn.preprocessing import LabelEncoder

    # Control the logger about system memory allocation
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

    # 乱数シードを設定
    np.random.seed(0)

    # Create label encoder
    encoder = LabelEncoder()

    # 画像情報を設定
    channels = 1
    height, width = 64, 64
    target_size = (height, width)
    input_shape = (height, width, channels)

    def augment_images_generator(images, labels):
        '''
        訓練用のデータの array (3d array) を受け取り, 要素である 2d array を基に拡張された 2d array を 3d array に詰め直して返す関数。
        新たにデータ拡張手法を追加したい場合は、以下の operations に lamda 関数を追加することで可能である。
        '''
        mu = 0
        sigma = 0.1
        operations = [
            # 実行されるデータ拡張手法
            lambda x: x,
            lambda x: cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE),
            lambda x: cv2.rotate(x, cv2.ROTATE_180),
            lambda x: cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE),
            lambda x: cv2.flip(x, 0),
            lambda x: cv2.flip(cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE), 0),
            lambda x: cv2.flip(cv2.rotate(x, cv2.ROTATE_180), 0),
            lambda x: cv2.flip(cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE), 0),
      ]

        # should this be processed with multi threading?
        for i in range(8):
            yield np.array([random.choice(operations)((img - np.min(img)) / (np.max(img) - np.min(img))) for img in images]).reshape(-1, height, width, channels), labels

    # オリジナルの画像をリサイズ
    resized_images = [ cv2.resize(cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, (0,0,0)), target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in train_df['waferMap'] ]

    # ターゲットをワンホットエンコード
    # ラベルの準備
    encoded_Y = encoder.fit_transform(train_df['failureType'])
    y_train = to_categorical(encoded_Y)
    number_of_classes = y_train.shape[1]

    # ニューラルネットワークの作成を開始
    model = Sequential()

    # 64フィルタ、5x5の窓を持ち、活性化関数としてReLUを用いるコンボリューション層を追加
    model.add(Conv2D(filters=64,
                    kernel_size=(5, 5),
                    input_shape=input_shape,
                    activation='relu'))

    # 2x2の窓を持つプーリング層を追加
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # ドロップアウト層を追加
    model.add(Dropout(0.5))

    # 入力をベクトルにする層を追加
    model.add(Flatten())

    # 活性化関数としてReLUを用いる128ユニットの全結合層を追加
    model.add(Dense(128, activation="relu"))

    # ドロップアウト層を追加
    model.add(Dropout(0.5))

    # 活性化関数としてソフトマックス関数を用いる全結合層を追加
    model.add(Dense(number_of_classes, activation="softmax"))

    # ニューラルネットワークをコンパイル
    model.compile(loss="categorical_crossentropy", # クロスエントロピ
                    optimizer="adam", # Adam
                    metrics=["accuracy"]) # 性能指標は精度

    # ニューラルネットワークの構成を表示
    model.summary()

    # ニューラルネットワークを訓練
    for batch_X_train, batch_y_train in augment_images_generator(resized_images, y_train):
        model.fit(batch_X_train, batch_y_train, epochs=8, batch_size=500, validation_split=0.2)

    resized_images_test = [ cv2.resize(cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, (0,0,0)), target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in x_test_df['waferMap'] ]
    X_test = np.array([ (img - np.min(img)) / (np.max(img) - np.min(img)) for img in resized_images_test ]).reshape(-1, height, width, channels)
    del resized_images_test

    y_pred = np.argmax(model.predict(X_test), axis=-1)

    return pd.DataFrame({'failureType': encoder.inverse_transform(y_pred)}, index=x_test_df.index)

solution関数は以下のように活用され、平均精度を計算します。

In [ ]:
%%timeit -r 1 -n 1

# データのインポート
df=pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = (matching_rows/(len(y_test_df_by_failure_type)))
    
    average_accuracy = sum(accuracies.values())/len(accuracies)

print(f"平均精度：{average_accuracy*100:.2f}%")

2023-11-22 03:13:43.072486: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 30, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 30, 30, 64)        0         
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 128)               7372928   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        